<a href="https://colab.research.google.com/github/mahn-bonnie/Robotics/blob/main/PID_Racetrack_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# --------------
# User Instructions
#
# Define a function cte in the robot class that will
# compute the crosstrack error for a robot on a
# racetrack with a shape as described in the video.
#
# You will need to base your error calculation on
# the robot's location on the track. Remember that
# the robot will be traveling to the right on the
# upper straight segment and to the left on the lower
# straight segment.
#
# --------------
# Grading Notes
#
# We will be testing your cte function directly by
# calling it with different robot locations and making
# sure that it returns the correct crosstrack error.

from math import *
import random


# ------------------------------------------------
#
# this is the robot class
#

class robot:

    # --------
    # init:
    #    creates robot and initializes location/orientation to 0, 0, 0
    #

    def __init__(self, length = 20.0):
        self.x = 0.0
        self.y = 0.0
        self.orientation = 0.0
        self.length = length
        self.steering_noise = 0.0
        self.distance_noise = 0.0
        self.steering_drift = 0.0

    # --------
    # set:
    #    sets a robot coordinate
    #

    def set(self, new_x, new_y, new_orientation):

        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation) % (2.0 * pi)


    # --------
    # set_noise:
    #    sets the noise parameters
    #

    def set_noise(self, new_s_noise, new_d_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.steering_noise = float(new_s_noise)
        self.distance_noise = float(new_d_noise)

    # --------
    # set_steering_drift:
    #    sets the systematical steering drift parameter
    #

    def set_steering_drift(self, drift):
        self.steering_drift = drift

    # --------
    # move:
    #    steering = front wheel steering angle, limited by max_steering_angle
    #    distance = total distance driven, most be non-negative

    def move(self, steering, distance,
             tolerance = 0.001, max_steering_angle = pi / 4.0):

        if steering > max_steering_angle:
            steering = max_steering_angle
        if steering < -max_steering_angle:
            steering = -max_steering_angle
        if distance < 0.0:
            distance = 0.0


        # make a new copy
        res = robot()
        res.length         = self.length
        res.steering_noise = self.steering_noise
        res.distance_noise = self.distance_noise
        res.steering_drift = self.steering_drift

        # apply noise
        steering2 = random.gauss(steering, self.steering_noise)
        distance2 = random.gauss(distance, self.distance_noise)

        # apply steering drift
        steering2 += self.steering_drift

        # Execute motion
        turn = tan(steering2) * distance2 / res.length

        if abs(turn) < tolerance:

            # approximate by straight line motion

            res.x = self.x + (distance2 * cos(self.orientation))
            res.y = self.y + (distance2 * sin(self.orientation))
            res.orientation = (self.orientation + turn) % (2.0 * pi)

        else:

            # approximate bicycle model for motion

            radius = distance2 / turn
            cx = self.x - (sin(self.orientation) * radius)
            cy = self.y + (cos(self.orientation) * radius)
            res.orientation = (self.orientation + turn) % (2.0 * pi)
            res.x = cx + (sin(res.orientation) * radius)
            res.y = cy - (cos(res.orientation) * radius)

        return res




    def __repr__(self):
        return '[x=%.5f y=%.5f orient=%.5f]'  % (self.x, self.y, self.orientation)


############## ONLY ADD / MODIFY CODE BELOW THIS LINE ####################
    def cte(self, radius):
        cte = 0.
        if self.y > radius and self.x <= 3 * radius and self.x >= radius:
            cte = self.y - (2*radius)

        elif self.y < radius and self.x <= 3 * radius and self.x >= radius:
            cte = -self.y

        elif self.x <= radius:
            sy = self.y - radius
            sx = self.x - radius
            theta = atan2(sy,sx)

            x1 = radius * cos(theta)
            y1 = radius * sin(theta)
            lrobot = sqrt(sy*sy + sx*sx)
            lpath = sqrt(x1*x1 + y1*y1)
            cte = lrobot - lpath

        elif self.x >= 3 * radius:
            sy = self.y-radius
            sx = self.x-(3*radius)
            theta = atan2(sy,sx)

            x1 = radius * cos(theta)
            y1 = radius * sin(theta)
            lrobot = sqrt(sy*sy + sx*sx)
            lpath = sqrt(x1*x1 + y1*y1)
            cte = lrobot - lpath

        return cte

############## ONLY ADD / MODIFY CODE ABOVE THIS LINE ####################




# ------------------------------------------------------------------------
#
# run - does a single control run.


def run(params, radius, printflag = False):
    myrobot = robot()
    myrobot.set(0.0, radius, pi / 2.0)
    speed = 1.0 # motion distance is equal to speed (we assume time = 1)
    err = 0.0
    int_crosstrack_error = 0.0
    N = 200

    crosstrack_error = myrobot.cte(radius) # You need to define the cte function!

    for i in range(N*2):
        diff_crosstrack_error = - crosstrack_error
        crosstrack_error = myrobot.cte(radius)
        diff_crosstrack_error += crosstrack_error
        int_crosstrack_error += crosstrack_error
        steer = - params[0] * crosstrack_error \
                - params[1] * diff_crosstrack_error \
                - params[2] * int_crosstrack_error
        myrobot = myrobot.move(steer, speed)
        if i >= N:
            err += crosstrack_error ** 2
        if printflag:
            print( myrobot)
    return err / float(N)

radius = 25.0
params = [10.0, 15.0, 0]
err = run(params, radius, True)
print ('\nFinal paramaeters: ', params, '\n ->', err)


[x=0.00000 y=26.00000 orient=1.57080]
[x=0.01365 y=26.99988 orient=1.54349]
[x=0.06592 y=27.99840 orient=1.49349]
[x=0.16804 y=28.99307 orient=1.44349]
[x=0.31973 y=29.98139 orient=1.39349]
[x=0.52064 y=30.96090 orient=1.34349]
[x=0.77025 y=31.92914 orient=1.29349]
[x=1.06793 y=32.88369 orient=1.24349]
[x=1.41296 y=33.82218 orient=1.19349]
[x=1.80445 y=34.74224 orient=1.14349]
[x=2.24145 y=35.64159 orient=1.09349]
[x=2.71993 y=36.51960 orient=1.05016]
[x=3.22162 y=37.38464 orient=1.04034]
[x=3.72989 y=38.24584 orient=1.03489]
[x=4.25610 y=39.09613 orient=0.99837]
[x=4.81855 y=39.92283 orient=0.94837]
[x=5.42162 y=40.72039 orient=0.89837]
[x=6.06380 y=41.48681 orient=0.84837]
[x=6.74348 y=42.22017 orient=0.79837]
[x=7.45896 y=42.91865 orient=0.74837]
[x=8.19955 y=43.59058 orient=0.72529]
[x=8.95301 y=44.24807 orient=0.70967]
[x=9.72345 y=44.88548 orient=0.67270]
[x=10.52084 y=45.48877 orient=0.62270]
[x=11.34739 y=46.05146 orient=0.57270]
[x=12.20102 y=46.57213 orient=0.52270]
[x=13.078